In [9]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import entropy
import matplotlib.pyplot as plt
import os
from joblib import Parallel, delayed

# 确保导入你上传的模块
from partitions import Partition
from system2 import RationalModelSystem2

In [ ]:

# ==========================================
# 1. 定义混合模型 (Metacognitive Arbitrator)
# ==========================================

class MetacognitiveArbitrator:
    """
    元认知仲裁器：管理 System 1 (Rule) 和 System 2 (Exemplar/Cluster) 的交互
    """
    def __init__(self, 
                 n_dims=4, 
                 n_cats=2, 
                 beta_s1=5.0,        # S1 温度
                 arb_threshold=2.0,  # 熵阈值
                 arb_slope=5.0,      # 仲裁斜率 (固定或拟合)
                 hint_alpha=0.5,     # S2 -> S1 提示强度 (固定或拟合)
                 s2_params=None):
        
        # === System 1: Hypotheses Testing ===
        self.s1_core = Partition(n_dims, n_cats)
        self.n_hypos = self.s1_core.length
        self.s1_posterior = np.ones(self.n_hypos) / self.n_hypos # 初始均匀分布
        self.beta_s1 = beta_s1
        self.s1_centers = self.s1_core.get_centers() 

        # === System 2: Rational Model (Anderson) ===
        if s2_params is None:
            s2_params = {'alpha': 1.0, 'sigma': 0.15, 'l0': 0.001}
        self.s2_core = RationalModelSystem2(n_dims, **s2_params)

        # === Arbitration Parameters ===
        self.arb_threshold = arb_threshold
        self.arb_slope = arb_slope
        self.hint_alpha = hint_alpha
        
        # === History Logs ===
        self.history = {
            'w_s1': [], 'entropy': [], 
            'p_correct': [], 'human_correct': []
        }

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def get_s1_entropy(self):
        return entropy(self.s1_posterior)

    def calculate_arbitration_weight(self):
        """计算依赖 System 1 的权重 lambda"""
        H = self.get_s1_entropy()
        # 熵越低(H < threshold)，S1 越可信，weight 接近 1
        weight = self._sigmoid(-self.arb_slope * (H - self.arb_threshold))
        return weight, H

    def _compute_s2_hint_signal(self):
        """S2 -> S1: 基于 Cluster 结构提示可能的规则"""
        s2_centers = [c['sum_x']/c['N'] for c in self.s2_core.clusters]
        if not s2_centers:
            return np.ones(self.n_hypos)

        hint_vector = np.zeros(self.n_hypos)
        for h in range(self.n_hypos):
            # 获取该假设下的理论中心
            theory_centers = np.array(list(self.s1_centers[h][1].values()))
            
            # 计算 S2 中心与该规则理论中心的匹配度
            total_dist = 0
            for mu in s2_centers:
                dists = np.linalg.norm(theory_centers - mu, axis=1)
                total_dist += np.min(dists) ** 2
            
            avg_dist = total_dist / len(s2_centers)
            hint_vector[h] = np.exp(-5.0 * avg_dist) # 距离越近，Hint 越强

        return hint_vector

    def predict(self, stimulus, return_details=False):
        # 1. System 1 预测
        # 构造 dummy data 以调用 Partition 接口
        dummy_data = ([stimulus], [1], [1])
        s1_liks = np.zeros((self.s1_core.n_cats, self.n_hypos))
        for h in range(self.n_hypos):
            # 获取 P(y|x, h)
            prob = self.s1_core.calc_likelihood_base(h, dummy_data, beta=self.beta_s1)
            s1_liks[:, h] = prob[:, 0]
        p_s1 = np.dot(s1_liks, self.s1_posterior) # 贝叶斯平均
        
        # 2. System 2 预测
        p_s2_dict = self.s2_core.get_choice_probs(stimulus)
        p_s2 = np.array([p_s2_dict.get(c+1, 0) for c in range(self.s1_core.n_cats)])
        
        # 3. 仲裁融合
        w_s1, entropy_val = self.calculate_arbitration_weight()
        p_final = w_s1 * p_s1 + (1 - w_s1) * p_s2
        p_final /= np.sum(p_final) # 归一化
        
        if return_details:
            return p_final, {'w_s1': w_s1, 'p_s1': p_s1, 'p_s2': p_s2, 'entropy': entropy_val}
        return p_final

    def update(self, stimulus, category):
        cat_idx = int(category) - 1
        
        # 1. 更新 S1 后验
        dummy_data = ([stimulus], [1], [1])
        likelihoods = np.zeros(self.n_hypos)
        for h in range(self.n_hypos):
            prob = self.s1_core.calc_likelihood_base(h, dummy_data, beta=self.beta_s1)
            likelihoods[h] = prob[cat_idx, 0]
        
        self.s1_posterior *= likelihoods
        if np.sum(self.s1_posterior) > 0:
            self.s1_posterior /= np.sum(self.s1_posterior)
        else:
            self.s1_posterior = np.ones(self.n_hypos) / self.n_hypos
            
        # 2. 更新 S2
        self.s2_core.update(stimulus, category)
        
        # 3. S2 -> S1 提示 (Bottom-up Hint)
        # 仅当主要依赖 S2 (w_s1 低) 且 S2 已经积累了一些经验时触发
        w_s1, _ = self.calculate_arbitration_weight()
        if w_s1 < 0.8 and self.s2_core.total_N > 5:
            hint_vector = self._compute_s2_hint_signal()
            self.s1_posterior *= (1 + self.hint_alpha * hint_vector)
            if np.sum(self.s1_posterior) > 0:
                self.s1_posterior /= np.sum(self.s1_posterior)

In [10]:
class MetacognitiveArbitrator:
    def __init__(self, 
                 n_dims=4, 
                 n_cats=2, 
                 beta_s1=5.0, 
                 arb_threshold=2.0, 
                 arb_slope=5.0, 
                 hint_alpha=0.5, 
                 s2_params=None,
                 s1_core=None): # [优化] 新增参数，允许传入预计算好的 S1
        
        # === System 1: Hypotheses Testing ===
        # [优化] 如果传入了现成的 s1_core，就直接用，避免重复计算几何分割
        if s1_core is not None:
            self.s1_core = s1_core
        else:
            self.s1_core = Partition(n_dims, n_cats)
            
        self.n_hypos = self.s1_core.length
        self.s1_posterior = np.ones(self.n_hypos) / self.n_hypos
        self.beta_s1 = beta_s1
        self.s1_centers = self.s1_core.get_centers() 
        
        # === System 2: Rational Model ===
        if s2_params is None:
            s2_params = {'alpha': 1.0, 'sigma': 0.15, 'l0': 0.001}
        self.s2_core = RationalModelSystem2(n_dims, **s2_params)
        
        # === Arbitration Parameters ===
        self.arb_threshold = arb_threshold
        self.arb_slope = arb_slope
        self.hint_alpha = hint_alpha
        
        self.history = {'w_s1': [], 'entropy': [], 'prob_correct': [], 'human_correct': []}

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def get_s1_entropy(self):
        return entropy(self.s1_posterior)

    def calculate_arbitration_weight(self):
        H = self.get_s1_entropy()
        weight = self._sigmoid(-self.arb_slope * (H - self.arb_threshold))
        return weight, H

    def _compute_s2_hint_signal(self):
        """计算 Hint (已包含之前的 Index 修复)"""
        s2_centers = []
        s2_labels = []
        for cluster in self.s2_core.clusters:
            mu = cluster['sum_x'] / cluster['N']
            pred_cat = max(cluster['y_counts'], key=cluster['y_counts'].get)
            s2_centers.append(mu)
            s2_labels.append(pred_cat - 1)

        if not s2_centers:
            return np.ones(self.n_hypos)

        s2_centers = np.array(s2_centers) 
        n_clusters = len(s2_centers)
        hint_vector = np.zeros(self.n_hypos)

        dummy_data = (s2_centers, [1]*n_clusters, [1]*n_clusters)

        for h in range(self.n_hypos):
            # calc_likelihood_boundary 返回 shape: [n_cats, n_trials]
            prob_matrix = self.s1_core.calc_likelihood_boundary(h, dummy_data, beta=self.beta_s1)
            # 取出每个 cluster (trial) 对应其预测类别 (cat) 的概率
            cluster_match_scores = prob_matrix[s2_labels, np.arange(n_clusters)]
            hint_vector[h] = np.mean(cluster_match_scores)

        return hint_vector

    def predict(self, stimulus, return_details=False):
        # 1. S1 预测
        dummy_data = ([stimulus], [1], [1])
        s1_liks = np.zeros((self.s1_core.n_cats, self.n_hypos))
        
        for h in range(self.n_hypos):
            prob_matrix = self.s1_core.calc_likelihood_boundary(h, dummy_data, beta=self.beta_s1)
            s1_liks[:, h] = prob_matrix[:, 0] # [修复] 正确的切片

        p_s1 = np.dot(s1_liks, self.s1_posterior)
        
        # 2. S2 预测
        p_s2_dict = self.s2_core.get_choice_probs(stimulus)
        p_s2 = np.array([p_s2_dict.get(c+1, 0) for c in range(self.s1_core.n_cats)])
        
        # 3. 仲裁
        w_s1, entropy_val = self.calculate_arbitration_weight()
        p_final = w_s1 * p_s1 + (1 - w_s1) * p_s2
        p_final /= np.sum(p_final)
        
        if return_details:
            return p_final, {'w_s1': w_s1, 'p_s1': p_s1, 'p_s2': p_s2, 'entropy': entropy_val}
        return p_final

    def update(self, stimulus, category):
        cat_idx = int(category) - 1
        
        # 1. S1 更新
        dummy_data = ([stimulus], [1], [1])
        likelihoods = np.zeros(self.n_hypos)
        
        for h in range(self.n_hypos):
            prob_matrix = self.s1_core.calc_likelihood_boundary(h, dummy_data, beta=self.beta_s1)
            likelihoods[h] = prob_matrix[cat_idx, 0] # [修复] 正确的切片
        
        self.s1_posterior *= likelihoods
        if np.sum(self.s1_posterior) > 0:
            self.s1_posterior /= np.sum(self.s1_posterior)
        else:
            self.s1_posterior = np.ones(self.n_hypos) / self.n_hypos
            
        # 2. S2 更新
        self.s2_core.update(stimulus, category)
        
        # 3. Hint 更新
        w_s1, _ = self.calculate_arbitration_weight()
        if w_s1 < 0.8 and self.s2_core.total_N > 5:
            hint_vector = self._compute_s2_hint_signal()
            self.s1_posterior *= (1 + self.hint_alpha * hint_vector)
            if np.sum(self.s1_posterior) > 0:
                self.s1_posterior /= np.sum(self.s1_posterior)

In [11]:
def neg_log_likelihood(params, df_sub, s1_core_shared):
    """
    NLL 计算函数
    s1_core_shared: 预先初始化好的 Partition 对象，避免重复计算
    """
    beta_s1, arb_threshold, s2_alpha, s2_sigma = params
    
    # 传递 s1_core_shared 进模型
    model = MetacognitiveArbitrator(
        n_dims=4, n_cats=2,
        beta_s1=beta_s1,
        arb_threshold=arb_threshold,
        arb_slope=5.0, 
        hint_alpha=0.5,
        s2_params={'alpha': s2_alpha, 'sigma': s2_sigma, 'l0': 0.001},
        s1_core=s1_core_shared # 关键优化
    )
    
    nll = 0.0
    for _, row in df_sub.iterrows():
        x = np.array([row['feature1'], row['feature2'], row['feature3'], row['feature4']])
        choice = int(row['choice'])
        true_cat = int(row['category'])
        
        probs = model.predict(x)
        p_choice = probs[choice-1]
        p_choice = max(p_choice, 1e-6)
        nll -= np.log(p_choice)
        model.update(x, true_cat)
        
    return nll

def fit_single_subject(sub_id, sub_df):
    """
    单个被试的完整拟合流程，将被 Parallel 调用
    """
    try:
        print(f"[Start] Subject {sub_id}")
        
        # 在每个进程内初始化一次 s1_core，供该进程内的 minimize 多次调用
        # 虽然每个进程都会初始化一次 Partition，但比 minimize 每次迭代都初始化要快几百倍
        s1_core_local = Partition(n_dims=4, n_cats=2)
        
        # 初始参数 [beta_s1, arb_threshold, s2_alpha, s2_sigma]
        x0 = [5.0, 2.0, 1.0, 0.15]
        bounds = [(0.1, 20), (0.1, 5), (0.1, 5), (0.01, 0.5)]
        
        # 运行优化
        res = minimize(
            neg_log_likelihood, 
            x0, 
            args=(sub_df, s1_core_local), # 传入 s1_core
            bounds=bounds, 
            method='L-BFGS-B'
        )
        
        # 记录参数结果
        param_result = {
            'iSub': sub_id,
            'beta_s1': res.x[0],
            'arb_threshold': res.x[1],
            's2_alpha': res.x[2],
            's2_sigma': res.x[3],
            'nll': res.fun
        }
        
        # === 重新运行一次以获取历史数据 (用于画图) ===
        best_model = MetacognitiveArbitrator(
            n_dims=4, n_cats=2,
            beta_s1=res.x[0],
            arb_threshold=res.x[1],
            arb_slope=5.0, hint_alpha=0.5,
            s2_params={'alpha': res.x[2], 'sigma': res.x[3], 'l0': 0.001},
            s1_core=s1_core_local
        )
        
        history = {'iSub': [], 'trial': [], 'w_s1': [], 'entropy': [], 'prob_correct': [], 'human_correct': []}
        for i, row in sub_df.iterrows():
            x = np.array([row['feature1'], row['feature2'], row['feature3'], row['feature4']])
            true_cat = int(row['category'])
            choice = int(row['choice'])
            
            probs, details = best_model.predict(x, return_details=True)
            best_model.update(x, true_cat)
            
            history['iSub'].append(sub_id)
            history['trial'].append(i+1)
            history['w_s1'].append(details['w_s1'])
            history['entropy'].append(details['entropy'])
            history['prob_correct'].append(probs[true_cat-1])
            history['human_correct'].append(1 if choice == true_cat else 0)
        
        print(f"[Done] Subject {sub_id} (NLL: {res.fun:.2f})")
        return param_result, pd.DataFrame(history)
        
    except Exception as e:
        print(f"[Error] Subject {sub_id}: {e}")
        return None, None

# ==========================================
# 3. 主程序：并行调度
# ==========================================

if __name__ == "__main__":
    
    if not os.path.exists('df_con1.csv'):
        print("错误：找不到 df_con1.csv")
    else:
        df = pd.read_csv('df_con1.csv')
        sub_ids = df['iSub'].unique()
        
        os.makedirs('plots_hybrid', exist_ok=True)
        
        # === 并行执行 ===
        # n_jobs=-1 表示使用所有可用的 CPU 核心
        print(f"开始并行拟合 {len(sub_ids)} 个被试，使用 CPU 核心数: {os.cpu_count()}...")
        
        results_list = Parallel(n_jobs=-1)(
            delayed(fit_single_subject)(sub, df[df['iSub'] == sub].copy().reset_index(drop=True))
            for sub in sub_ids
        )
        
        # === 整理结果 ===
        all_params = []
        all_histories = []
        
        for param, hist in results_list:
            if param is not None:
                all_params.append(param)
                all_histories.append(hist)
        
        # 保存参数
        params_df = pd.DataFrame(all_params)
        params_df.to_csv('fitted_params_hybrid_v2.csv', index=False)
        print("拟合完成，参数已保存。")
        
        # === 绘图 (串行绘图即可，速度很快) ===
        print("开始生成绘图...")
        for hist_df in all_histories:
            sub = hist_df['iSub'].iloc[0]
            
            fig, ax1 = plt.subplots(figsize=(10, 6))
            
            # 左轴：权重
            color = 'tab:blue'
            ax1.set_xlabel('Trial')
            ax1.set_ylabel('Reliance on Rule (Weight S1)', color=color, fontweight='bold')
            ax1.plot(hist_df['trial'], hist_df['w_s1'], color=color, linewidth=2, label='Weight S1')
            ax1.tick_params(axis='y', labelcolor=color)
            ax1.set_ylim(-0.05, 1.05)
            
            # 右轴：熵
            ax2 = ax1.twinx()
            color = 'tab:red'
            ax2.set_ylabel('Rule Uncertainty (S1 Entropy)', color=color, fontweight='bold')
            ax2.plot(hist_df['trial'], hist_df['entropy'], color=color, linestyle='--', linewidth=2, label='Entropy')
            ax2.tick_params(axis='y', labelcolor=color)
            
            plt.title(f'Subject {sub}: Cognitive Control Dynamics')
            fig.tight_layout()
            plt.savefig(f'plots_hybrid_v2/sub_{sub}_dynamics.png')
            plt.close()
            
        print("所有任务完成！")

开始并行拟合 8 个被试，使用 CPU 核心数: 128...


[Start] Subject 22
[Start] Subject 7
[Start] Subject 16
[Start] Subject 1
[Start] Subject 19
Loading similarity matrix from disk: /home/yangjiong/CategoryLearning_gitcode/src/Hybrid/cache/similarity_matrix_d4_c2.npy
[Start] Subject 4
Loading similarity matrix from disk: /home/yangjiong/CategoryLearning_gitcode/src/Hybrid/cache/similarity_matrix_d4_c2.npy
Loading similarity matrix from disk: /home/yangjiong/CategoryLearning_gitcode/src/Hybrid/cache/similarity_matrix_d4_c2.npy
Loading similarity matrix from disk: /home/yangjiong/CategoryLearning_gitcode/src/Hybrid/cache/similarity_matrix_d4_c2.npy
Loading similarity matrix from disk: /home/yangjiong/CategoryLearning_gitcode/src/Hybrid/cache/similarity_matrix_d4_c2.npy
Loading similarity matrix from disk: /home/yangjiong/CategoryLearning_gitcode/src/Hybrid/cache/similarity_matrix_d4_c2.npy
[Start] Subject 10
Loading similarity matrix from disk: /home/yangjiong/CategoryLearning_gitcode/src/Hybrid/cache/similarity_matrix_d4_c2.npy
[Start] S